In [1]:
pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LambdaCallback

# Introducción
En este dataset, se analizan una cantidad de imágenes, esto se consiguió en la propia librería de TenserFlow, ya que al tratar de usar las de kaggle son demasiados ejemplos por lo que el archivo se vuelve muy pesado. El dataset trata de clasificar las imágenes, en los siguientes campos, aviones, autos, pájaros, gatos, venados, perros, rana, caballos, barco y camiones. Eso en cuanto a las neuronas convulcionales, después lo que se requiere es usar el Data Aumentation, para crear mucho más imágenes, variando multiples factores de esta como la posición, desplazar y rotar. El uso de Data Augmentation no solo aumenta el tamaño del conjunto de datos sino que también introduce variabilidad, lo que permite que el modelo sea más robusto y generalice mejor en datos no vistos.
# Objetivos del Notebook
- Crear un modelo de CNN
- Ver el impacto del Data Augmentation en el modelo
- Saber interpretar los resultados en ambos casos del modelo


In [3]:
# Cargar el dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalizar las imágenes a valores entre 0 y 1 para que se pueda hacer el entrenamiento y pruebas en el modelo, luego de está limpieza
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


In [4]:
# En el dataset se puede encontro 50000 imágenes que están distribuidas en las 10 categorías anteriores de 32x32 y a color
print(train_images.shape)

(50000, 32, 32, 3)


In [5]:
# Crear un modelo
model = models.Sequential()

# Primera capa convolucional: 32 filtros de 3x3, activación ReLU, con imágenes de entrada de 32x32x3
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Segunda capa convolucional con 64 filtros
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Tercera capa convolucional con 64 filtros
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Aplanar la salida para las capas densas
model.add(layers.Flatten())

# Agregar una capa densa con 64 neuronas
model.add(layers.Dense(64, activation='relu'))

# Capa de salida con 10 neuronas (correspondiente a las 10 clases de CIFAR-10), usando softmax
model.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Crear un callback para mostrar precisión y error por época
metrics_callback = LambdaCallback(on_epoch_end=lambda epoch, logs:
                                  print(f'Epoch {epoch + 1}: Precisión = {logs["accuracy"] * 100:.2f}%, Error = {(1 - logs["accuracy"]) * 100:.2f}%'))

# Entrenar el modelo sin Data Augmentation
print("Entrenamiento sin Data Augmentation")
history_no_augmentation = model.fit(train_images, train_labels,
                                    epochs=10,
                                    batch_size=64,
                                    callbacks=[metrics_callback])

# Evaluar el modelo sin Data Augmentation
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'\nTest accuracy sin Data Augmentation: {test_acc}')

# Guardar los pesos iniciales del modelo antes de reiniciarlo
initial_weights = model.get_weights()

# Reiniciar los pesos del modelo
model.set_weights(initial_weights)

Entrenamiento sin Data Augmentation
Epoch 1/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.3222 - loss: 1.8468Epoch 1: Precisión = 41.20%, Error = 58.80%
782/782 ━━━━━━━━━━━━━━━━━━━━ 69s 86ms/step - accuracy: 0.3224 - loss: 1.8462
Epoch 2/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5253 - loss: 1.3204Epoch 2: Precisión = 54.26%, Error = 45.74%
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 87ms/step - accuracy: 0.5253 - loss: 1.3203
Epoch 3/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5919 - loss: 1.1526Epoch 3: Precisión = 60.40%, Error = 39.60%
782/782 ━━━━━━━━━━━━━━━━━━━━ 80s 85ms/step - accuracy: 0.5920 - loss: 1.1525
Epoch 4/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.6440 - loss: 1.0183Epoch 4: Precisión = 64.61%, Error = 35.39%
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 84ms/step - accuracy: 0.6440 - loss: 1.0183
Epoch 5/10
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.6707 - loss: 0.9484Epoch 5: Precisión = 67.06%, Error = 32.94%
782/78

In [7]:
# Crear un generador de imágenes con data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,       # Rotar las imágenes aleatoriamente hasta 20 grados
    width_shift_range=0.2,   # Desplazamiento horizontal de hasta el 20% de la imagen
    height_shift_range=0.2,  # Desplazamiento vertical de hasta el 20% de la imagen
    horizontal_flip=True,    # Voltear horizontalmente las imágenes
)

# Ajustar el generador de datos al conjunto de entrenamiento
datagen.fit(train_images)

In [8]:
# Entrenar el modelo con Data Augmentation
print("Entrenamiento con Data Augmentation")
history_with_augmentation = model.fit(datagen.flow(train_images, train_labels, batch_size=64),
                                      steps_per_epoch=len(train_images) // 64,
                                      epochs=10,
                                      validation_data=(test_images, test_labels),
                                      callbacks=[metrics_callback])

# Evaluar el modelo con Data Augmentation
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'\nTest accuracy con Data Augmentation: {test_acc}')

Entrenamiento con Data Augmentation
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.5548 - loss: 1.2674Epoch 1: Precisión = 56.57%, Error = 43.43%
781/781 ━━━━━━━━━━━━━━━━━━━━ 104s 133ms/step - accuracy: 0.5548 - loss: 1.2673 - val_accuracy: 0.6688 - val_loss: 0.9604
Epoch 2/10
  1/781 ━━━━━━━━━━━━━━━━━━━━ 55s 71ms/step - accuracy: 0.5156 - loss: 1.5350

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Epoch 2: Precisión = 51.56%, Error = 48.44%
781/781 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5156 - loss: 1.5350 - val_accuracy: 0.6754 - val_loss: 0.9455
Epoch 3/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.5878 - loss: 1.1700Epoch 3: Precisión = 59.11%, Error = 40.89%
781/781 ━━━━━━━━━━━━━━━━━━━━ 101s 129ms/step - accuracy: 0.5878 - loss: 1.1700 - val_accuracy: 0.6652 - val_loss: 0.9630
Epoch 4/10
  1/781 ━━━━━━━━━━━━━━━━━━━━ 55s 71ms/step - accuracy: 0.5312 - loss: 1.3071Epoch 4: Precisión = 53.12%, Error = 46.88%
781/781 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.5312 - loss: 1.3071 - val_accuracy: 0.6721 - val_loss: 0.9391
Epoch 5/10
781/781 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.5999 - loss: 1.1339Epoch 5: Precisión = 60.39%, Error = 39.61%
781/781 ━━━━━━━━━━━━━━━━━━━━ 106s 135ms/step - accuracy: 0.5999 - loss: 1.1339 - val_accuracy: 0.6699 - val_loss: 0.9665
Epoch 6/10
  1/781 ━━━━━━━━━━━━━━━━━━━━ 53s 69ms/step - accuracy: 0.6406 - loss: 1.1410

# Conclusión e interpretación
Se observa que el modelo alcanzó un 69.60% de precisión en las imágenes de prueba cuando se utilizó Data Augmentation, mientras que sin esta técnica el resultado fue ligeramente mejor, con un 70.10% de precisión. Este comportamiento puede explicarse por la naturaleza del dataset, en el cual las categorías son visualmente distintas entre sí, ya que son aviones, perros, caballos, edificios, camiones, carros... entre otros.

El aumento de ejemplos mediante Data Augmentation podría haber introducido variaciones en las imágenes que no ayudaron a mejorar el rendimiento del modelo, ya que este no tenía dificultades significativas para distinguir entre las categorías originales. De hecho, el exceso de imágenes modificadas podría haber añadido complejidad innecesaria al entrenamiento, provocando una ligera disminución en la precisión final.

En contextos donde las imágenes son más similares, por ejemplo, distinguir entre perros y lobos, el uso de Data Augmentation podría haber sido más beneficioso, ya que ayudaría al modelo a generalizar mejor. Sin embargo en este ejemplo, como las categorias son tan diferentes esto no ayudo a que generalizada mejor, sino que lo hizo un poco más ineficiente, pero teniendo buenos resultados en ambos casos.

AL final de todo esto el Data Augmentation puede ayudar significativamente a crear más ejemplos para que el modelo de redes CNN pueda generalizar mejor, pero en imagenes que si pueda existir confusión, mientras que en este modelo, con categorias tan distintas, no lo ayudo demasiado a pesar de que de nuevo dieroin buenos resultados ambos modelos